# Chapter 8 機械学習の基礎（教師あり学習）

# 8-1 機械学習の全体像

## 8-1-1 機械学習とは

### 1-1 教師あり学習と教師なし学習

### 1-2 強化学習

### 1-3 機械学習を適用するにあたって

## 8-1-2 教師あり学習

### 2-1 教師あり学習の手法

## 8-1-3 教師なし学習

### 3-1 教師なし学習の手法

## 8-1-4 強化学習

### 4-1 強化学習の手法

## 8-1-5 この章で使用するライブラリのインポート

In [16]:
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd

# 可視化ライブラリ
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 機械学習ライブラリ
import sklearn

%precision 3

'%.3f'

# 8-2 重回帰

## 8-2-1 自動車価格データの取り込み

In [17]:
import requests, zipfile
import io

# 自動車価格データを取得
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
res = requests.get(url).content

# 取得したデータをDataFrameオブジェクトとして読み込み
auto = pd.read_csv(io.StringIO(res.decode('utf-8')), header=None)

# データの列にラベルを設定
auto.columns=['symboling', 'normalized-loss', 'make', 'fuel-type', 'aspiration', 'num-of-doors',
			  'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height',
			  'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore',
			  'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']

In [18]:
print('自動車データの形式:{}'.format(auto.shape))

自動車データの形式:(205, 26)


In [19]:
auto.head()

,symboling,normalized-loss,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


## 8-2-2 データの整理
### 2-1 不適切なデータの除去

In [20]:
# それぞれのカラムに ? がいくつあるかカウント
auto = auto[['price', 'horsepower', 'width', 'height']]
auto.isin(['?']).sum()

price         4
horsepower    2
width         0
height        0
dtype: int64

In [21]:
# ? をNaNに置き換えて、NaNがある行を削除
auto = auto.replace('?', np.nan).dropna()
print('自動車データの形式:{}'.format(auto.shape))

自動車データの形式:(199, 4)


### 2-2 型の変換

In [22]:
print('データ型の確認（型変換前）\n{}\n'.format(auto.dtypes))

データ型の確認（型変換前）
price          object
horsepower     object
width         float64
height        float64
dtype: object



In [23]:
# objectをnumericに変換！
auto = auto.assign(price=pd.to_numeric(auto.price))
auto = auto.assign(horsepower=pd.to_numeric(auto.horsepower))
print('データ型の確認（型変換後）\n{}\n'.format(auto.dtypes))

データ型の確認（型変換後）
price           int64
horsepower      int64
width         float64
height        float64
dtype: object



### 2-3 相関の確認

In [25]:
# 相関係数を確認。
# priceが今回の目的変数
auto.corr()

,price,horsepower,width,height
price,1.000000,0.810533,0.753871,0.134990
horsepower,0.810533,1.000000,0.615315,-0.087407
width,0.753871,0.615315,1.000000,0.309223
height,0.134990,-0.087407,0.309223,1.000000


price(目的変数)以外に着目すると、widthとhorsepowerの相関が0.6とやや高め。実務では多重共線性が生じる可能性があるので、重回帰モデル構築においては、総監の高い変数軍からは代表の変数しか使用しない。（ここでは気にしていないが）

## 8-2-3 モデル構築と評価

In [27]:
# データ分割のためのインポート
from sklearn.model_selection import train_test_split

# 重回帰のモデル構築のためのインポート
from sklearn.linear_model import LinearRegression

# 目的変数にpriceを指定、説明変数にそれ以外を指定
X = auto.drop('price', axis=1)
y = auto['price']

# 訓練データとテストデータを分ける
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# 重回帰クラスの初期化と学習
model = LinearRegression()
model.fit(X_train, y_train)

# 決定係数を表示
print('決定係数(train):{:, .3f}'.format(model.score(X_train, y_train)))
print('決定係数(test):{:, .3f}'.format(model.score(X_test, y_test)))

# 回帰係数と切片を表示
print('\n回帰係数\n{}'.format(pd.Series(model.coef_, index=X.columns)))
print('切片:{:, 3f}'.format(model.intercept_))

ValueError: Invalid format specifier ', .3f' for object of type 'float'